# `Layers` in TensorFlow-Slim
*by Marvin Bertin*
<img src="../images/tensorflow.png" width="400">

### TensorFlow-Slim - `Layers`
TF-Slim provides a set of  high level layers that makes building models in tensorflow simpler and more concise. <br>
All layer related functions are accesssible in [layers.py](https://github.com/tensorflow/tensorflow/blob/eb56a8af24695bf8258addf28b0c53fbabff72e6/tensorflow/contrib/layers/python/layers/layers.py). <br>
<br>
<img src="../images/Conv2.png" width="400">
<center>A 2D Convolutional Neural Network</center>

The following methods will be covered in this notebook:
```
slim.conv2d
slim.fully_connected
slim.max_pool2d
slim.dropout
slim.softmax
slim.repeat
slim.stack
```

## Import TensorFlow

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
slim = tf.contrib.slim

## Import Helper Functions

In [2]:
import sys  
sys.path.append("../") 

from utils.pretty_printer import inspect_variables, inspect_layers

## Convolutional and Maxpool Layers with Native TensorFlow
Neural Network layers often consist of multiple variables and tensor operations combined together. For example a single convolutional layer takes in two variables( a weight tensor and bias vector) and combines them with a 2D convolution operation followed by a non-linear activation function.

### Convolutional Layer
<br>
<div style="float:left;margin-right:5px;">
    <img src="../images/Conv3.jpeg" width="300" />
    <p style="text-align:center;">*2D Convolution on color image*</p>
</div>
<div style="float:center;margin-right:5px;">
    <img src="../images/neuron_model.jpeg" width="350" />
    <p style="text-align:center;">*A Neural Network "neuron"*</p>
</div>

<img src="../images/Convolution.gif" width="400">
<center>Convolution with 3×3 Filter</center>

**Want to learn more?**
- [Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io/convolutional-networks/)
- [Feature Extraction Using Convolution](http://ufldl.stanford.edu/tutorial/supervised/FeatureExtractionUsingConvolution/)
- [Understanding Convolutions](http://colah.github.io/posts/2014-07-Understanding-Convolutions/)
- [Conv Nets: A Modular Perspective](http://colah.github.io/posts/2014-07-Conv-Nets-Modular/)
### Maxpool Layer
<br>
<div style="float:left;margin-right:5px;">
    <img src="../images/pool.jpeg" width="300" />
    <p style="text-align:center;">*Spatial downsampling with filter size 2, stride 2*</p>
</div>
<div style="float:center;margin-right:5px;">
    <img src="../images/maxpool.jpeg" width="400" />
    <p style="text-align:center;">*Maxpooling operation*</p>
</div>

** Want to learn more?**
- [Pooling: Overview](http://ufldl.stanford.edu/tutorial/supervised/Pooling/)

### Native TensorFlow Code
This creates a lot of boilerplate code that quickly becomes unreadabe and unmaintanable beyound a few layers. <br>
Here is an example of a relativaly simple model combining 2 convolutional layers with a max-pooling layer.

In [3]:
# Initialize graph
g = tf.Graph()
with g.as_default():
    
    # 4D Tensor placeholder for input images
    inputs = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name="images")

    # Convolutional layers variable scope
    with tf.variable_scope("TF-Native-conv", [inputs]):
        # conv1
        with tf.variable_scope('conv1'):
            weights = tf.get_variable('weights',
                                      shape=[3, 3, 3, 64],
                                      initializer=tf.random_normal_initializer(
                                          stddev=0.1),
                                      trainable=True,
                                      dtype=tf.float32)
            conv = tf.nn.conv2d(inputs, weights, [1, 1, 1, 1], padding='SAME')
            biases = tf.get_variable('biases',
                                     shape=[64],
                                     initializer=tf.constant_initializer(0.0),
                                     trainable=True,
                                     dtype=tf.float32)
            out = tf.nn.bias_add(conv, biases)
            conv1 = tf.nn.relu(out, name="relu")

        # conv2
        with tf.variable_scope('conv2'):
            weights = tf.get_variable('weights',
                                      shape=[3, 3, 64, 64],
                                      initializer=tf.random_normal_initializer(
                                          stddev=0.1),
                                      trainable=True,
                                      dtype=tf.float32)
            conv = tf.nn.conv2d(conv1, weights, [1, 1, 1, 1], padding='SAME')
            biases = tf.get_variable('biases',
                                     shape=[64],
                                     initializer=tf.constant_initializer(0.0),
                                     trainable=True,
                                     dtype=tf.float32)
            out = tf.nn.bias_add(conv, biases)
            conv2 = tf.nn.relu(out, name="relu")

        # pool3
        with tf.variable_scope('pool3'):
            outputs = tf.nn.max_pool(conv2,
                                     ksize=[1, 2, 2, 1],
                                     strides=[1, 2, 2, 1],
                                     padding='SAME',
                                     name='max_pool')

## Inspect Variables

In [4]:
with g.as_default():
    print("Parameters:")
    inspect_variables(slim.get_variables(scope="TF-Native-conv"))
    print("Inputs/Outputs:")
    inspect_variables([inputs, outputs])

Parameters:
name = TF-Native-conv/conv1/weights:0                          shape = (3, 3, 3, 64)
name = TF-Native-conv/conv1/biases:0                           shape = (64,)
name = TF-Native-conv/conv2/weights:0                          shape = (3, 3, 64, 64)
name = TF-Native-conv/conv2/biases:0                           shape = (64,)

Inputs/Outputs:
name = images:0                                                shape = (?, 32, 32, 3)
name = TF-Native-conv/pool3/max_pool:0                         shape = (?, 16, 16, 64)



## Convolutional and Maxpool Layers with TensorFlow-Slim
TF-Slim provides high level layer abstractions that removes boilerplate code while remaining flexible. <br>
<br>
For example `slim.conv2d` combines all these low level operations in one function:

- Creates the weight and bias variables
- Convolves the weights with the input from the previous layer
- Adds the biases to the result of the convolution.
- Applies an activation function.
---
    => slim.conv2d(*args, **kwargs)
    Docstring:
    Adds a 2D convolution followed by an optional batch_norm layer.

    Args:
      inputs: a 4-D tensor  `[batch_size, height, width, channels]`.
      num_outputs: integer, the number of output filters.
      kernel_size: a list of length 2 `[kernel_height, kernel_width]` of
        of the filters.
      stride: a list of length 2 `[stride_height, stride_width]`.
      padding: one of `VALID` or `SAME`.
      activation_fn: activation function, set to None to skip it and maintain
        a linear activation.
      normalizer_fn: normalization function to use instead of `biases`. If
        `normalizer_fn` is provided then `biases_initializer` and
        `biases_regularizer` are ignored and `biases` are not created nor added.
        default set to None for no normalizer function
      normalizer_params: normalization function parameters.
      weights_initializer: An initializer for the weights.
      weights_regularizer: Optional regularizer for the weights.
      biases_initializer: An initializer for the biases. If None skip biases.
      biases_regularizer: Optional regularizer for the biases.
      scope: Optional scope for `variable_scope`.
      ...

    Returns:
      a tensor representing the output of the operation.

### Code with TF-Slim

In [5]:
g = tf.Graph()
with g.as_default():
    
    # 4D Tensor placeholder for input images
    inputs = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name="images")

    with tf.variable_scope("TF-Slim-conv", [inputs]):
        # conv1
        net = slim.conv2d(inputs, 64, [3, 3],
                          padding='SAME',
                          weights_initializer = tf.random_normal_initializer(stddev=0.1),
                          scope='conv1')
        # conv2
        net = slim.conv2d(net, 64, [3, 3],
                          padding='SAME',
                          weights_initializer = tf.random_normal_initializer(stddev=0.1),
                          scope='conv2')
        # pool3
        outputs = slim.max_pool2d(net, [2, 2], scope = "pool3")    

## Inspect Variables

In [6]:
with g.as_default():
    print("Parameters:")
    inspect_variables(slim.get_variables(scope="TF-Slim-conv"))
    print("Inputs/Outputs:")
    inspect_variables([inputs, outputs])

Parameters:
name = TF-Slim-conv/conv1/weights:0                            shape = (3, 3, 3, 64)
name = TF-Slim-conv/conv1/biases:0                             shape = (64,)
name = TF-Slim-conv/conv2/weights:0                            shape = (3, 3, 64, 64)
name = TF-Slim-conv/conv2/biases:0                             shape = (64,)

Inputs/Outputs:
name = images:0                                                shape = (?, 32, 32, 3)
name = TF-Slim-conv/pool3/MaxPool:0                            shape = (?, 16, 16, 64)



## Meta-operation `slim.repeat`
TF-Slim also provides two meta-operations called `slim.repeat` and `slim.stack` that allow to repeat the same operation multiple times and reduce the code size even further.

    => slim.repeat(inputs, repetitions, layer, *args, **kwargs)
    Docstring:
    Applies the same layer with the same arguments repeatedly.

    Args:
      inputs: A `Tensor` suitable for layer.
      repetitions: Int, number of repetitions.
      layer: A layer with arguments `(inputs, *args, **kwargs)`
      *args: Extra args for the layer.
      **kwargs: Extra kwargs for the layer.

    Returns:
      a tensor result of applying the layer, repetitions times.

In [7]:
g = tf.Graph()
with g.as_default():
    
    inputs = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name="images")

    with tf.variable_scope("TF-Slimer-conv", [inputs]):
        # conv 1-2
        net = slim.repeat(inputs, 2,
                          slim.conv2d,
                          64, [3, 3],
                          padding='SAME',
                          weights_initializer = tf.random_normal_initializer(stddev=0.1),
                          scope='conv')
        # pool
        outputs = slim.max_pool2d(net, [2, 2], scope='pool3')  

## Inspect Variables

In [8]:
with g.as_default():
    print("Parameters:")
    inspect_variables(slim.get_variables(scope="TF-Slimer-conv"))
    print("Inputs/Outputs:")
    inspect_variables([inputs, outputs])

Parameters:
name = TF-Slimer-conv/conv/conv_1/weights:0                    shape = (3, 3, 3, 64)
name = TF-Slimer-conv/conv/conv_1/biases:0                     shape = (64,)
name = TF-Slimer-conv/conv/conv_2/weights:0                    shape = (3, 3, 64, 64)
name = TF-Slimer-conv/conv/conv_2/biases:0                     shape = (64,)

Inputs/Outputs:
name = images:0                                                shape = (?, 32, 32, 3)
name = TF-Slimer-conv/pool3/MaxPool:0                          shape = (?, 16, 16, 64)



## Fully Connected Layers in Naive TensorFlow
A feedforward (fully connected) layer is the most basic layer in neural networks.
<div style="float:right;margin-right:5px;">
    <img src="../images/SingleNeuron.png" width="300" />
    <p style="text-align:center;">*Single feedforward neuron*</p>
</div>
<br>
**Feedforward computation**

$\textstyle h_{W,b}(x) = f(W^Tx) = f(\sum_{i=1}^3 W_{i}x_i +b)$ <br>

$f =$ activation function <br>
$W =$ weight vector/matrix <br>
$b =$ bias scalar/vector <br>

**Want to learn more?**
- [Multi-Layer Neural Network](http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/)
- [Neural Networks Demystified](https://www.youtube.com/watch?v=bxe2T-V8XRs&list=PLiaHhY2iBX9hdHaRr6b7XevZtgZRa1PoU) (video series)
- [Neural Network in Python](https://github.com/MarvinBertin/NNet/blob/master/Neural%20Net.ipynb) (from scratch)
### Multi-layer Neural Network
<br>
<img src="../images/fc.jpeg" width="500">
### Native TensorFlow Code
Although this neural network is very simple, the tensorflow code is quite lengthy and repetitive. The neural network ends with a dropout layer and a softmax layer.
You can learn more about these layers below.
<img src="../images/dropout1.png" width="600">

**Want to learn more?**
- [Deep learning - Dropout](https://www.youtube.com/watch?v=UcKPdAM8cnI)
- [Softmax Regression](http://ufldl.stanford.edu/tutorial/supervised/SoftmaxRegression/)
- [Linear Classification](http://cs231n.github.io/linear-classify/)

In [9]:
g = tf.Graph()
with g.as_default():
    # 2D tensor placeholder for input data (512 features)
    inputs = tf.placeholder(tf.float32, shape=[None, 512], name="inputs")
    
    with tf.variable_scope('TF-Native-fc', [inputs]):

        # fc1
        with tf.variable_scope('fc1'):
            # weight matrix
            fcw = tf.get_variable('weights',
                                  shape=[512, 1024],
                                  initializer=tf.random_normal_initializer(stddev=0.1),
                                  trainable=True,
                                  dtype=tf.float32) 
            # bias vector
            fcb = tf.get_variable('biases',
                                  shape=[1024],
                                  initializer=tf.constant_initializer(1.0),
                                  trainable=True,
                                  dtype=tf.float32)
            # dot product with bias term
            fcl = tf.nn.bias_add(tf.matmul(inputs, fcw), fcb)
            # activation function
            fc = tf.nn.relu(fcl)
        
        # fc2
        with tf.variable_scope('fc2'):
            fcw = tf.get_variable('weights',
                                  shape=[1024, 256],
                                  initializer=tf.random_normal_initializer(stddev=0.1),
                                  trainable=True,
                                  dtype=tf.float32) 

            fcb = tf.get_variable('biases',
                                 shape=[256],
                                 initializer=tf.constant_initializer(1.0),
                                 trainable=True,
                                 dtype=tf.float32)
            fcl = tf.nn.bias_add(tf.matmul(fc, fcw), fcb)
            fc = tf.nn.relu(fcl)
        
        # fc3
        with tf.variable_scope('fc3'):
            fcw = tf.get_variable('weights',
                                  shape=[256, 10],
                                  initializer=tf.random_normal_initializer(stddev=0.1),
                                  trainable=True,
                                  dtype=tf.float32) 

            fcb = tf.get_variable('biases',
                                 shape=[10],
                                 initializer=tf.constant_initializer(1.0),
                                 trainable=True,
                                 dtype=tf.float32)
            fcl = tf.nn.bias_add(tf.matmul(fc, fcw), fcb)
            fc = tf.nn.relu(fcl)
        
        # dropout
        with tf.variable_scope('dropout4'):
            fc = tf.nn.dropout(fc, 0.5, name='dropout')
        
        # softmax
        with tf.variable_scope('softmax5'):
            outputs = tf.nn.softmax(fc, name='softmax')

## Inspect Variables

In [10]:
with g.as_default():
    print("Parameters:")
    inspect_variables(slim.get_variables(scope="TF-Native-fc"))
    print("Inputs/Outputs:")
    inspect_variables([inputs, outputs])

Parameters:
name = TF-Native-fc/fc1/weights:0                              shape = (512, 1024)
name = TF-Native-fc/fc1/biases:0                               shape = (1024,)
name = TF-Native-fc/fc2/weights:0                              shape = (1024, 256)
name = TF-Native-fc/fc2/biases:0                               shape = (256,)
name = TF-Native-fc/fc3/weights:0                              shape = (256, 10)
name = TF-Native-fc/fc3/biases:0                               shape = (10,)

Inputs/Outputs:
name = inputs:0                                                shape = (?, 512)
name = TF-Native-fc/softmax5/softmax:0                         shape = (?, 10)



## Fully Connected Layers in TensorFlow-Slim
TF-slim provides you with a fully connected layer implementation that abstracts the layer very similarly to the convolutional layer we worked with earlier.

    => slim.fully_connected(*args, **kwargs)
    Docstring:
    Adds a fully connected layer.

    Args:
      inputs: A tensor of with at least rank 2 and value for the last dimension,
        i.e. `[batch_size, depth]`.
      num_outputs: Integer or long, the number of output units in the layer.
      activation_fn: activation function, set to None to skip it and maintain
        a linear activation.
      normalizer_fn: normalization function to use instead of `biases`. If
        `normalizer_fn` is provided then `biases_initializer` and
        `biases_regularizer` are ignored and `biases` are not created nor added.
        default set to None for no normalizer function
      normalizer_params: normalization function parameters.
      weights_initializer: An initializer for the weights.
      weights_regularizer: Optional regularizer for the weights.
      biases_initializer: An initializer for the biases. If None skip biases.
      biases_regularizer: Optional regularizer for the biases.
      scope: Optional scope for variable_scope.
      ...

    Returns:
       the tensor variable representing the result of the series of operations.

### Code with TF-Slim

In [11]:
g = tf.Graph()
with g.as_default():
    inputs = tf.placeholder(tf.float32, shape=[None, 512], name="inputs")

    with tf.variable_scope('TF-Slim-fc', [inputs]):
        # fc1
        net = slim.fully_connected(inputs, 1024,
                                   weights_initializer = tf.random_normal_initializer(
                                    stddev=0.1),
                                   scope='fc1')
        # fc2
        net = slim.fully_connected(net, 256,
                                   weights_initializer = tf.random_normal_initializer(
                                    stddev=0.1),
                                   scope='fc2')
        # fc3
        net = slim.fully_connected(net, 10,
                                   weights_initializer = tf.random_normal_initializer(
                                    stddev=0.1),
                                   scope='fc3')
        # dropout
        net = slim.dropout(net, 0.5, scope='dropout4')
        # softmax
        outputs = slim.softmax(net, scope='softmax5')

## Inspect Variables

In [12]:
with g.as_default():
    print("Parameters:")
    inspect_variables(slim.get_variables(scope="TF-Slim-fc"))
    print("Inputs/Outputs:")
    inspect_variables([inputs, outputs])

Parameters:
name = TF-Slim-fc/fc1/weights:0                                shape = (512, 1024)
name = TF-Slim-fc/fc1/biases:0                                 shape = (1024,)
name = TF-Slim-fc/fc2/weights:0                                shape = (1024, 256)
name = TF-Slim-fc/fc2/biases:0                                 shape = (256,)
name = TF-Slim-fc/fc3/weights:0                                shape = (256, 10)
name = TF-Slim-fc/fc3/biases:0                                 shape = (10,)

Inputs/Outputs:
name = inputs:0                                                shape = (?, 512)
name = TF-Slim-fc/softmax5/Reshape_1:0                         shape = (?, 10)



## Meta-operation `slim.stack`

    => slim.stack(inputs, layer, stack_args, **kwargs)
    Docstring:
    Builds a stack of layers by applying layer repeatedly using stack_args.
    
    Args:
      inputs: A `Tensor` suitable for layer.
      layer: A layer with arguments `(inputs, *args, **kwargs)`
      stack_args: A list/tuple of parameters for each call of layer.
      **kwargs: Extra kwargs for the layer.

    Returns:
      a `Tensor` result of applying the stacked layers.
      
### Code with TF-Slim

In [13]:
g = tf.Graph()
with g.as_default():
    inputs = tf.placeholder(tf.float32, shape=[None, 512], name="inputs")

    with tf.variable_scope('TF-Slim-fc', [inputs]):
        # fc1-3
        net = slim.stack(inputs, slim.fully_connected, [1024, 256, 10],
                         weights_initializer = tf.random_normal_initializer(stddev=0.1),
                         scope='fc')
        # dropout
        net = slim.dropout(net, 0.5, scope='dropout4')
        # softmax
        outputs = slim.softmax(net, scope='softmax5')

## Inspect Variables

In [14]:
with g.as_default():
    print("Parameters:")
    inspect_variables(slim.get_variables(scope="TF-Slim-fc"))
    print("Inputs/Outputs:")
    inspect_variables([inputs, outputs])

Parameters:
name = TF-Slim-fc/fc/fc_1/weights:0                            shape = (512, 1024)
name = TF-Slim-fc/fc/fc_1/biases:0                             shape = (1024,)
name = TF-Slim-fc/fc/fc_2/weights:0                            shape = (1024, 256)
name = TF-Slim-fc/fc/fc_2/biases:0                             shape = (256,)
name = TF-Slim-fc/fc/fc_3/weights:0                            shape = (256, 10)
name = TF-Slim-fc/fc/fc_3/biases:0                             shape = (10,)

Inputs/Outputs:
name = inputs:0                                                shape = (?, 512)
name = TF-Slim-fc/softmax5/Reshape_1:0                         shape = (?, 10)



## Next Lesson
### `Scope` in TensorFlow-Slim
-  Define default arguments for specific operations within a scope

<img src="../images/divider.png" width="100">